In [ ]:
import GPy
import numpy as np
import safeopt
import os
import logging
from multiprocessing import Process, Pipe, Lock, connection
from typing import List, Tuple, Dict, Final, Callable

# logging.basicConfig(filename="app.log", filemode="w", level=logging.DEBUG)
# logging.basicConfig(format="%(process)d-%(levelname)s-%(message)s")

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
# Create handlers
f_handler = logging.FileHandler("file.log", "w")
# f_handler.setLevel(logging.INFO)
# Create formatters and add it to handlers
f_format = logging.Formatter(
    "%(process)d - %(levelname)6s - %(funcName)20s() : %(message)s"
)
f_handler.setFormatter(f_format)
# Add handlers to the logger
logger.addHandler(f_handler)

all_parameter_subspaces = []
subspace_indices_for_hyperspace = []
subspaces_deployment_status = []
points_evaluated_in_hyperspace = {}
evaluation_constraint = 50
lock = Lock()

In [ ]:
# domain of the function
BIRD_FUNCTION_BOUNDS: Final = [(-2 * np.pi, 2 * np.pi), (-2 * np.pi, 2 * np.pi)]

# user defined threshold for function
BIRD_FUNCTION_THRESHOLD: Final = -35.0


def bird_function(X):
    """Bird function.
    -f* = 106.764537 at
    (x, y) = (4.70104, 3.15294) and
    (x, y) = (-1.58214, -3.13024)
    """
    X = np.atleast_2d(X)

    x = X[:, 0]
    y = X[:, 1]
    noise = np.random.normal(0, 0.5 ** 2)
    F = (
        np.sin(y) * np.exp((1 - np.cos(x)) ** 2)
        + np.cos(x) * np.exp((1 - np.sin(y)) ** 2)
        + (x - y) ** 2
        + noise
    )
    # optimization involves finding maximum value of function
    return -F.reshape((-1, 1))


In [ ]:
def create_hyperspaces(
    parameter_spaces: List[Tuple], no_subspaces: int
) -> List[List[Tuple]]:
    """Creates hyperspaces for the given parameters by dividing each of them into given number of subspaces.

    Args:
        parameter_spaces:
            List of tuple containing two items i.e: lower bound and upper bound for each search parameter.
        no_subspaces:
            How many number of subspaces to create for a search parameter.

    Returns:
        hyperspaces:
            Set of all possible combinations of subspaces.
    """
    global subspaces_deployment_status
    global all_parameter_subspaces
    global subspace_indices_for_hyperspace
    # to divide each parameter space into given number of subspaces
    for parameter_space in parameter_spaces:
        low, high = parameter_space
        subspace_length = abs(high - low) / no_subspaces
        parameter_subspaces = []
        for i in range(no_subspaces):
            end = low + subspace_length
            parameter_subspaces.append((round(low, 8), round(end, 8)-0.00000001))
            low = end
        all_parameter_subspaces.append(parameter_subspaces)

    rows = len(all_parameter_subspaces)  # no_parameters
    columns = len(all_parameter_subspaces[0])  # no_subspaces

    # initializing deployed status for each subsapce
    for i in range(rows):
        each_parameter_subspaces = []
        for _ in range(columns):
            # for each subspace maintain these 3 states to determine its deployment status
            # [is this subspace deployed (True/False), associated with any subspace (True/False),
            #  hyperspace number (None-not deployed with any hyperspace/int-deployed with that hyperspace)]
            each_parameter_subspaces.append(False)
        subspaces_deployment_status.append(each_parameter_subspaces)

    # no_hyperspaces = no_subspaces ** no_parameters
    hyperspaces = []  # contains all possible combinations of subspaces
    for _ in range(columns ** rows):
        hyperspaces.append([])
        subspace_indices_for_hyperspace.append([])

    for row in range(rows):
        repeat = columns ** (rows - row - 1)
        for column in range(columns):
            item = all_parameter_subspaces[row][column]
            start = column * repeat
            for times in range(columns ** row):
                for l in range(repeat):
                    hyperspaces[start + l].append(item)
                    subspace_indices_for_hyperspace[start + l].append(column)
                start += columns * repeat
    return hyperspaces

In [ ]:
def which_hyperspace(x: List[List], hyperspaces: List[List[Tuple]]) -> Dict:
    """Creates a dictionary with hyperspace number as key and list of points belong to that hyperspace as
    corresponding value.

    Args:
        x:
            List of points.
        hyperspaces:
            List containing all hyperspaces.

    Returns:
        safe_hyperspaces:
            Dictionary with hyperspace number and list of points as key-value pair.
    """
    safe_hyperspaces = {}

    for point in x:
        for i, hyperspace in enumerate(hyperspaces):
            belongs_flag = True
            for dimension, value in enumerate(point):
                if value >= hyperspace[dimension][0] and value <= hyperspace[dimension][1]:
                    continue
                belongs_flag = False
            if belongs_flag:
                if i not in safe_hyperspaces:
                    safe_hyperspaces[i] = [point]
                else:
                    safe_hyperspaces[i].append(point)
    return safe_hyperspaces

In [ ]:
bounds = BIRD_FUNCTION_BOUNDS
no_subspaces = 4
bounds_indices = [(0, no_subspaces - 1) for _ in range(len(bounds))]
safe_threshold = BIRD_FUNCTION_THRESHOLD
# parameter_set = safeopt.linearly_spaced_combinations(bounds, 1000)
hyperspaces = create_hyperspaces(bounds, no_subspaces)

In [ ]:
which_hyperspace([[3.1415926400000003,  6.2831853]], hyperspaces)

In [ ]:
def get_bounds_from_index(hyperspace_bounds: List[Tuple]) -> List[Tuple]:
    """ """
    bounds = []
    for parameter_no, bound in enumerate(hyperspace_bounds):
        parameter_subspaces = all_parameter_subspaces[parameter_no]
        low = parameter_subspaces[bound[0]][0]
        high = parameter_subspaces[bound[1]][1]
        bounds.append((low, high))
    return bounds

In [ ]:
get_bounds_from_index(bounds_indices)